<a href="https://colab.research.google.com/github/Rizm10/UK-HPI-streamlit-Dashboard/blob/main/HPI_UK_improved_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [4]:
!pip install pyngrok


In [17]:
with open("app.py", "w") as f:
    f.write('''
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image

# --- Page Setup ---
st.set_page_config(page_title="UK Housing Dashboard", layout="wide")

# --- Sidebar Logo & Filters ---
logo = Image.open("gov_logo.png")
st.sidebar.image(logo, use_column_width=True)
st.sidebar.markdown("<hr style='border:1px solid #1D70B8;'/>", unsafe_allow_html=True)
st.sidebar.header("Filters")

# --- Info Box ---
st.markdown(
    """
    <div style='background-color:#F3F2F1;padding:1rem;border-left:5px solid #1D70B8;border-radius:5px;margin-bottom:1rem'>
    <strong>ℹ️ About This Dashboard</strong><br>
    This dashboard provides key insights into the UK House Price Index, based on official data updated monthly.
    </div>
    """,
    unsafe_allow_html=True
)

# --- Load Data ---
df = pd.read_csv("UK-HPI-full-file-2025-02.csv")
df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)
df = df.dropna(subset=["Date"]).sort_values("Date")

# --- Sidebar Filters ---
regions = st.sidebar.multiselect("Select Region(s)", options=df["RegionName"].unique(), default=["London", "South East"])
property_types = st.sidebar.multiselect("Select Property Type(s)", options=["Detached", "SemiDetached", "Terraced", "Flat"], default=["Detached", "Flat"])
date_range = st.sidebar.date_input("Select Date Range", [df["Date"].min(), df["Date"].max()])
price_min, price_max = st.sidebar.slider("Select Price Range", int(df["AveragePrice"].min()), int(df["AveragePrice"].max()), (int(df["AveragePrice"].min()), int(df["AveragePrice"].max())))

# --- Apply Filters ---
price_cols = {
    "Detached": "DetachedPrice",
    "SemiDetached": "SemiDetachedPrice",
    "Terraced": "TerracedPrice",
    "Flat": "FlatPrice"
}

filtered_df = df[
    (df["RegionName"].isin(regions)) &
    (df["Date"] >= pd.to_datetime(date_range[0])) &
    (df["Date"] <= pd.to_datetime(date_range[1])) &
    (df["AveragePrice"] >= price_min) &
    (df["AveragePrice"] <= price_max)
]

# --- KPI Summary ---
latest_date = filtered_df["Date"].max()
latest_df = filtered_df[filtered_df["Date"] == latest_date]
latest_price = latest_df["AveragePrice"].mean()
latest_volume = latest_df["SalesVolume"].sum()
prev_month = latest_date - pd.DateOffset(months=1)
prev_df = filtered_df[filtered_df["Date"] == prev_month]
prev_price = prev_df["AveragePrice"].mean() if not prev_df.empty else np.nan
pct_change = ((latest_price - prev_price) / prev_price) * 100 if not np.isnan(prev_price) else "N/A"

st.markdown("### 📊 Key Metrics")
col1, col2, col3, col4 = st.columns(4)
col1.metric("Latest Avg Price", f"£{latest_price:,.0f}")
col2.metric("Sales Volume", f"{int(latest_volume):,}")
col3.metric("MoM Change", f"{pct_change:.2f}%" if not isinstance(pct_change, str) else "N/A")
col4.metric("Last Updated", latest_date.strftime("%b %Y"))

# --- Charts Grid ---
c1, c2 = st.columns(2)

with c1:
    st.subheader("Monthly % Change in Avg Price")
    filtered_df = filtered_df.sort_values(by="Date")
    filtered_df["PctChange"] = filtered_df.groupby("RegionName")["AveragePrice"].pct_change() * 100
    fig1, ax1 = plt.subplots(figsize=(7, 4))
    for region in regions:
        region_data = filtered_df[filtered_df["RegionName"] == region]
        ax1.plot(region_data["Date"], region_data["PctChange"], label=region)
    ax1.axhline(0, color="gray", linestyle="--")
    ax1.set_ylabel("% Change")
    ax1.set_title("MoM % Change")
    ax1.legend()
    ax1.grid(True)
    st.pyplot(fig1)

with c2:
    st.subheader("Average Price Trend")
    fig2, ax2 = plt.subplots(figsize=(7, 4))
    for region in regions:
        region_data = filtered_df[filtered_df["RegionName"] == region]
        ax2.plot(region_data["Date"], region_data["AveragePrice"], label=region)
    ax2.set_xlabel("Date")
    ax2.set_ylabel("Average Price")
    ax2.set_title("House Prices Over Time")
    ax2.legend()
    ax2.grid(True)
    st.pyplot(fig2)

c3, c4 = st.columns(2)

with c3:
    st.subheader("Price Trend by Property Type")
    fig3, ax3 = plt.subplots(figsize=(7, 4))
    type_df = df[
        (df["RegionName"].isin(regions)) &
        (df["Date"] >= pd.to_datetime(date_range[0])) &
        (df["Date"] <= pd.to_datetime(date_range[1]))
    ]
    for p_type in property_types:
        ax3.plot(type_df["Date"], type_df[price_cols[p_type]], label=p_type)
    ax3.set_xlabel("Date")
    ax3.set_ylabel("Price")
    ax3.set_title("Avg Price by Property Type")
    ax3.legend()
    ax3.grid(True)
    st.pyplot(fig3)

with c4:
    st.subheader("Price Distribution")
    long_df = pd.melt(
        df[df["RegionName"].isin(regions)],
        id_vars=["Date", "RegionName"],
        value_vars=[price_cols[p] for p in property_types],
        var_name="Type", value_name="Price"
    )
    type_map = {v: k for k, v in price_cols.items()}
    long_df["Type"] = long_df["Type"].map(type_map)
    fig4, ax4 = plt.subplots(figsize=(7, 4))
    sns.histplot(data=long_df, x="Price", hue="Type", multiple="stack", bins=30, ax=ax4)
    ax4.set_xlabel("Price")
    ax4.set_title("Price Distribution by Type")
    ax4.grid(True)
    st.pyplot(fig4)

# --- Outlier Detection ---
Q1 = filtered_df["AveragePrice"].quantile(0.25)
Q3 = filtered_df["AveragePrice"].quantile(0.75)
IQR = Q3 - Q1
outliers = filtered_df[
    (filtered_df["AveragePrice"] < Q1 - 1.5 * IQR) |
    (filtered_df["AveragePrice"] > Q3 + 1.5 * IQR)
]

if not outliers.empty:
    with st.expander("📌 View Outliers"):
        st.markdown("### Potential Outliers in Average Price")
        st.dataframe(outliers[["Date", "RegionName", "AveragePrice"]].head(10))
''')


In [18]:
import subprocess
import time

# Start streamlit with subprocess (non-blocking)
process = subprocess.Popen(["streamlit", "run", "app.py"])
time.sleep(10)  # Wait for it to boot before ngrok connects

In [19]:
from pyngrok import ngrok
import time
import os
from google.colab import userdata

# Kill previous tunnels
ngrok.kill()

# Set ngrok auth token
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start streamlit in background
os.system("streamlit run app.py &")

# Give it time to boot
time.sleep(5)

# Connect ngrok to streamlit
public_url = ngrok.connect(8501)
print(f"🚀 Your app is live at: {public_url}")

🚀 Your app is live at: NgrokTunnel: "https://dc3134a53d22.ngrok-free.app" -> "http://localhost:8501"
